NOTES:
- I have no idea what I'm doing, but lets see how this goes
- I'm not going to bother quite yet with too much normalization
- Ok now gonna give on loading everything manually, just gonna use the pre loaded data set

In [1]:

import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from keras.datasets import cifar10
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input, Flatten, Dense


Using TensorFlow backend.


In [2]:
tf.test.is_gpu_available()

True

In [3]:
(trainX, trainY), (testX, testY) = cifar10.load_data()
print(trainX.shape, trainY.shape, testX.shape, testY.shape)
#one hot encode Y sets
testYmat=to_categorical(testY)
trainYmat=to_categorical(trainY)


(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [4]:
#normalize data to maximum
trainXnorm=trainX.astype('float32') / 255.0
testXnorm=testX.astype('float32') / 255.0

a basic 2 layer model 

In [6]:
def  two_layer_model():
    model = Sequential()
    model.add(Input((32,32,3)))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)




In [8]:
tl_model=two_layer_model()
tl_hist=tl_model.fit(trainXnorm, trainYmat, epochs=100, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 3s 63us/sample - loss: 1.9339 - accuracy: 0.3141 - val_loss: 1.8298 - val_accuracy: 0.3589
Epoch 2/100
50000/50000 [==============================] - 2s 46us/sample - loss: 1.7753 - accuracy: 0.3780 - val_loss: 1.7495 - val_accuracy: 0.3817
Epoch 3/100
50000/50000 [==============================] - 2s 46us/sample - loss: 1.7063 - accuracy: 0.4051 - val_loss: 1.6814 - val_accuracy: 0.4115
Epoch 4/100
50000/50000 [==============================] - 2s 46us/sample - loss: 1.6570 - accuracy: 0.4212 - val_loss: 1.6374 - val_accuracy: 0.4274
Epoch 5/100
50000/50000 [==============================] - 2s 46us/sample - loss: 1.6200 - accuracy: 0.4357 - val_loss: 1.6153 - val_accuracy: 0.4302
Epoch 6/100
50000/50000 [==============================] - 2s 46us/sample - loss: 1.5888 - accuracy: 0.4466 - val_loss: 1.5860 - val_accuracy: 0.4434
Epoch 7/100
50000/50000 [=========================

So this does not work very well, which is expected lets try something else,
a not so basic model, including a convolution and pool layers  

In [10]:
def basic_CNN():
    model = Sequential()
    model.add(Input((32,32,3)))
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', padding='same',))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [11]:
history = basic_CNN().fit(trainXnorm, trainYmat, epochs=100, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 16s 325us/sample - loss: 2.1468 - accuracy: 0.2038 - val_loss: 1.8844 - val_accuracy: 0.3289
Epoch 2/100
50000/50000 [==============================] - 13s 266us/sample - loss: 1.7293 - accuracy: 0.3795 - val_loss: 1.5448 - val_accuracy: 0.4419
Epoch 3/100
50000/50000 [==============================] - 13s 266us/sample - loss: 1.5088 - accuracy: 0.4613 - val_loss: 1.4078 - val_accuracy: 0.5043
Epoch 4/100
50000/50000 [==============================] - 13s 267us/sample - loss: 1.3900 - accuracy: 0.5040 - val_loss: 1.3499 - val_accuracy: 0.5192
Epoch 5/100
50000/50000 [==============================] - 13s 266us/sample - loss: 1.2995 - accuracy: 0.5367 - val_loss: 1.2730 - val_accuracy: 0.5433
Epoch 6/100
50000/50000 [==============================] - 13s 266us/sample - loss: 1.2158 - accuracy: 0.5677 - val_loss: 1.2005 - val_accuracy: 0.5731
Epoch 7/100
50000/50000 [=============

The accuracy of the basic CNN has converged to 1m but the validation accuracy is still pretty low, so the model is overfitting 
lets try adding a dropout regularization


In [5]:
def CNN_with_dropout():
    model = Sequential()
    model.add(Input((32,32,3)))
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',  padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [6]:
hist_dp=CNN_with_dropout().fit(trainXnorm, trainYmat, epochs=100, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 18s 363us/sample - loss: 2.2067 - accuracy: 0.1692 - val_loss: 2.0488 - val_accuracy: 0.2426
Epoch 2/100
50000/50000 [==============================] - 14s 273us/sample - loss: 1.9332 - accuracy: 0.2988 - val_loss: 1.8186 - val_accuracy: 0.3539
Epoch 3/100
50000/50000 [==============================] - 14s 274us/sample - loss: 1.7560 - accuracy: 0.3660 - val_loss: 1.6143 - val_accuracy: 0.4194
Epoch 4/100
50000/50000 [==============================] - 14s 274us/sample - loss: 1.6202 - accuracy: 0.4135 - val_loss: 1.4888 - val_accuracy: 0.4617
Epoch 5/100
50000/50000 [==============================] - 14s 274us/sample - loss: 1.5157 - accuracy: 0.4506 - val_loss: 1.3966 - val_accuracy: 0.4890
Epoch 6/100
50000/50000 [==============================] - 14s 274us/sample - loss: 1.4280 - accuracy: 0.4832 - val_loss: 1.3169 - val_accuracy: 0.5220
Epoch 7/100
50000/50000 [=============

So this helped a little with the validation los, but it still looks like the model is over fitting. Lets try changing the weight initialization. `he_normal` is supposed to to be good for relu activation functions, and change the optimizer to `adam` with defgault settings 

In [8]:
def CNN_dropout_he_initialization():
    model = Sequential()
    model.add(Input((32,32,3)))
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform' , padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [10]:
hist3=CNN_dropout_he_initialization().fit(trainXnorm, trainYmat, epochs=100, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 15s 307us/sample - loss: 1.5680 - accuracy: 0.4299 - val_loss: 1.1899 - val_accuracy: 0.5650
Epoch 2/100
50000/50000 [==============================] - 14s 282us/sample - loss: 1.0692 - accuracy: 0.6179 - val_loss: 0.9131 - val_accuracy: 0.6757
Epoch 3/100
50000/50000 [==============================] - 14s 282us/sample - loss: 0.8878 - accuracy: 0.6854 - val_loss: 0.8787 - val_accuracy: 0.6941
Epoch 4/100
50000/50000 [==============================] - 14s 282us/sample - loss: 0.7706 - accuracy: 0.7281 - val_loss: 0.7579 - val_accuracy: 0.7398
Epoch 5/100
50000/50000 [==============================] - 14s 282us/sample - loss: 0.6882 - accuracy: 0.7570 - val_loss: 0.6645 - val_accuracy: 0.7690
Epoch 6/100
50000/50000 [==============================] - 14s 282us/sample - loss: 0.6198 - accuracy: 0.7805 - val_loss: 0.6630 - val_accuracy: 0.7709
Epoch 7/100
50000/50000 [=============

looks like it got a littl stuck towards the end, gonna try and reduce number of epochs to 65, and add batch normalization

In [11]:
def CNN_dropout_he_initialization_batchnorm():
    model = Sequential()
    model.add(Input((32,32,3)))
    #conv block 1
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform' , padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 2
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 3
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Flatten())
    #dense layers 
    model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [12]:
hist4=CNN_dropout_he_initialization_batchnorm().fit(trainXnorm, trainYmat, epochs=65, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/65
50000/50000 [==============================] - 20s 394us/sample - loss: 1.3628 - accuracy: 0.5139 - val_loss: 1.0060 - val_accuracy: 0.6526
Epoch 2/65
50000/50000 [==============================] - 18s 353us/sample - loss: 0.8991 - accuracy: 0.6821 - val_loss: 0.8160 - val_accuracy: 0.7160
Epoch 3/65
50000/50000 [==============================] - 18s 352us/sample - loss: 0.7313 - accuracy: 0.7444 - val_loss: 0.7417 - val_accuracy: 0.7465
Epoch 4/65
50000/50000 [==============================] - 18s 352us/sample - loss: 0.6315 - accuracy: 0.7774 - val_loss: 0.6498 - val_accuracy: 0.7784
Epoch 5/65
50000/50000 [==============================] - 18s 352us/sample - loss: 0.5554 - accuracy: 0.8059 - val_loss: 0.6419 - val_accuracy: 0.7850
Epoch 6/65
50000/50000 [==============================] - 18s 352us/sample - loss: 0.5005 - accuracy: 0.8234 - val_loss: 0.5798 - val_accuracy: 0.8043
Epoch 7/65
50000/50000 [====================

looks like this helped a fair amount; lets see if adding an extra dense layer helps, and adding dropout and batch norm for the dense layers 

In [14]:
def CNN_extra_dense():
    model = Sequential()
    model.add(Input((32,32,3)))
    #conv block 1
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform' , padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 2
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 3
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Flatten())
    #dense layers 
    model.add(Dense(units=256, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [15]:
hist5=CNN_extra_dense().fit(trainXnorm, trainYmat, epochs=65, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/65
50000/50000 [==============================] - 21s 428us/sample - loss: 1.7720 - accuracy: 0.3969 - val_loss: 1.4734 - val_accuracy: 0.4875
Epoch 2/65
50000/50000 [==============================] - 19s 372us/sample - loss: 1.1689 - accuracy: 0.5846 - val_loss: 1.0227 - val_accuracy: 0.6415
Epoch 3/65
50000/50000 [==============================] - 19s 372us/sample - loss: 0.9598 - accuracy: 0.6682 - val_loss: 0.8498 - val_accuracy: 0.7033
Epoch 4/65
50000/50000 [==============================] - 19s 373us/sample - loss: 0.8376 - accuracy: 0.7115 - val_loss: 0.7050 - val_accuracy: 0.7577
Epoch 5/65
50000/50000 [==============================] - 19s 372us/sample - loss: 0.7495 - accuracy: 0.7413 - val_loss: 0.7610 - val_accuracy: 0.7381
Epoch 6/65
50000/50000 [==============================] - 19s 372us/sample - loss: 0.6794 - accuracy: 0.7702 - val_loss: 0.6623 - val_accuracy: 0.7725
Epoch 7/65
50000/50000 [====================

okay, so this tooks a little longer, but we got the best results so far. Now im gonna add a 4 convolutional block and see what happens 

In [22]:
def CNN_4_convs():
    model = Sequential()
    model.add(Input((32,32,3)))
    #conv block 1
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform' , padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 2
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 3
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 4
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    model.add(Flatten())
    #dense layers 
    model.add(Dense(units=256, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [24]:
hist7=CNN_4_convs().fit(trainXnorm, trainYmat, epochs=65, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/65
50000/50000 [==============================] - 28s 553us/sample - loss: 1.8477 - accuracy: 0.3659 - val_loss: 1.3917 - val_accuracy: 0.5049
Epoch 2/65
50000/50000 [==============================] - 24s 489us/sample - loss: 1.2173 - accuracy: 0.5671 - val_loss: 1.0281 - val_accuracy: 0.6312
Epoch 3/65
50000/50000 [==============================] - 24s 488us/sample - loss: 0.9729 - accuracy: 0.6631 - val_loss: 0.8518 - val_accuracy: 0.7035
Epoch 4/65
50000/50000 [==============================] - 24s 488us/sample - loss: 0.8290 - accuracy: 0.7197 - val_loss: 0.8041 - val_accuracy: 0.7317
Epoch 5/65
50000/50000 [==============================] - 24s 488us/sample - loss: 0.7243 - accuracy: 0.7555 - val_loss: 0.6741 - val_accuracy: 0.7694
Epoch 6/65
50000/50000 [==============================] - 24s 488us/sample - loss: 0.6489 - accuracy: 0.7823 - val_loss: 0.6751 - val_accuracy: 0.7734
Epoch 7/65
50000/50000 [====================

mariginally worse - lets try the previous version with an extra dense layer before softmaxing

In [31]:
def CNN_more_dense():
    model = Sequential()
    model.add(Input((32,32,3)))
    #conv block 1
    model.add(Conv2D(filters= 32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform' , padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 2
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.2))
    #conv block 3
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    #dense layers 
    model.add(Dense(units=256, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=64, activation='relu', kernel_initializer= 'he_uniform'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(Dense(units=10, activation='softmax')) 
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [32]:
hist9=CNN_more_dense().fit(trainXnorm, trainYmat, epochs=65, batch_size=64, validation_data=(testXnorm, testYmat), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/65
50000/50000 [==============================] - 22s 436us/sample - loss: 1.9705 - accuracy: 0.3151 - val_loss: 1.3437 - val_accuracy: 0.5134
Epoch 2/65
50000/50000 [==============================] - 19s 381us/sample - loss: 1.3718 - accuracy: 0.5042 - val_loss: 1.0662 - val_accuracy: 0.6175
Epoch 3/65
50000/50000 [==============================] - 19s 381us/sample - loss: 1.1487 - accuracy: 0.6015 - val_loss: 0.9085 - val_accuracy: 0.6803
Epoch 4/65
50000/50000 [==============================] - 19s 381us/sample - loss: 0.9946 - accuracy: 0.6639 - val_loss: 0.8286 - val_accuracy: 0.7122
Epoch 5/65
50000/50000 [==============================] - 19s 382us/sample - loss: 0.8915 - accuracy: 0.7030 - val_loss: 0.8110 - val_accuracy: 0.7265
Epoch 6/65
50000/50000 [==============================] - 19s 382us/sample - loss: 0.8119 - accuracy: 0.7335 - val_loss: 0.6745 - val_accuracy: 0.7684
Epoch 7/65
50000/50000 [====================

overall, this was worse, I think I have a good idea of how to go about this now though 